# SDS to website
Purpose: By processing a list of events from an Excel spreadsheet, look for existing seismic & infrasound data in an SDS archive, instrument correct those data, and make a webpage for each event 

Steps:
1. Create an event catalog with instrument-corrected data

TO DO: 
* add support for AM network, and any data from networks run by other operators and available on IRIS
* add well data from YYYYMMDD_final.pkl files

Do we want to turn pkl files into SDS?

# 1. Do full workflow for 1 particular event first
* Load and correct seismic and infrasound data from FL network and XA and whatever the network was for PASSCAL equipment

In [ ]:
import os
import sys
import header
paths = header.setup_environment()
paths['SDS_TOP'] = '/data/SDS'
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import obspy
sys.path.append(os.path.join(paths['Developer'], 'SoufriereHillsVolcano', 'lib'))
import SDS
import Spectrograms
import libWellData as LLE
#from obspy.clients.filesystem.sds import Client
from obspy.signal.cross_correlation import correlate, xcorr_max
%matplotlib inline
plt.ioff()

def dataframe2stream(df):
    stream = obspy.Stream()
    for column in df.columns:
        if column=='datetime':
            continue
        # Create Trace for each channel
        trace = obspy.Trace()
        trace.data = df[column].values
        if column=='AirPressureShallow':
            trace.id = '6S.Baro..HDD'
        elif column=='AirPressureDeep':
            trace.id = '6I.Baro..HDD'
        else:
            name = column.ljust(12, '_')
            if column in watercolumnsShallow:
                well = '6S'
                chan = 'WLM'
            elif column in watercolumnsIntermediate:
                well = '6I'
                chan = 'WLM'
            elif column in aircolumns:
                if column[-1]=='0':
                    well = '6S'
                else:
                    well = '6I'
                chan = 'HDD'
            trace.id = '.'.join((well, column[0:5], column[5:7], chan))
        if trace.stats.channel=='WLM':
            trace.stats['units'] = 'mh20'
        elif trace.stats.channel=='HDD':
            trace.stats['units'] = 'Pa'

        mintime = df['datetime'].min()
        try:
            trace.stats.starttime = obspy.UTCDateTime(mintime.timestamp())  # Set the start time
        except:
            print(mintime, df.loc[0,'datetime'])
            trace.stats.starttime = obspy.UTCDateTime(df.iloc[0]['datetime'].timestamp())
        trace.stats.delta = df['datetime'].diff().median().total_seconds()  # Sampling rate (1 Hz in this example)
        
        stream += trace  # Add the trace to the stream
    print(stream)
    return stream

from obspy.signal.trigger import coincidence_trigger
from pprint import pprint
import matplotlib.dates as dates
def detectEvent(st):
    st2 = st.copy().filter('bandpass', freqmin=5, freqmax=16)
    N = len(st)
    #trig = coincidence_trigger("recstalta", 3.5, 1, st, 3, sta=2, lta=40)
    num_triggers_needed = max(( int(N/2), 2)) # half the channels or 2, whichever is greater
    trig = coincidence_trigger("recstalta", 4, 1, st2, num_triggers_needed, sta=5, lta=100, max_trigger_length=180, delete_long_trigger=True, details=True)
    best_trig = {}
    best_product = 0

    for this_trig in trig:

        thistime = dates.date2num(this_trig['time'])
        this_product = this_trig['coincidence_sum']*this_trig['duration']
        if this_product > best_product:
            best_trig = this_trig
            best_product = this_product
    pprint(best_trig)
    return best_trig

def intersect(df, cols):
    common_cols = list(set(df.columns) & set(cols))
    return common_cols

paths['HTML_DIR'] = '/var/www/html/thompsong/KSC_EROSION/EVENTS'
paths['PNG_DIR'] = os.path.join(paths['HTML_DIR'], 'images')
paths['EVENT_WAVEFORMS'] = os.path.join(paths['local_outdir'], 'EVENTS') # must exist, and Excel file must be here
paths['csv_launches'] = os.path.join(paths['local_outdir'], 'PilotStudy_KSC_Rocket_Launches.csv')
#csv_launches_detected = os.path.join(paths['local_outdir'], 'PilotStudy_KSC_Rocket_Launches_detected.csv')
print(paths)
paths['welldata'] = os.path.join(paths['DROPBOX_DATA_TOP'], 'WellData', '05_DAILY')
launchesDF = LLE.removed_unnamed_columns(pd.read_csv(paths['csv_launches'], index_col=None))
launchesDF['datetime']= pd.to_datetime(launchesDF['Date'] + ' ' +  launchesDF['Time'])
preseconds=600
eventseconds=600
postseconds=600
aircolumns = ['AirPressureShallow', 'AirPressureDeep', '1226420', '1226429']
watercolumnsShallow = ['1226421', '1226419', '1226423']
watercolumnsIntermediate = ['2149882', '2151691', '2151692']
launchesDF['ontime']=None
launchesDF['offtime']=None
launchesDF['signalStrength']=None
launchesDF['Baro_amp']=None # units Pa
launchesDF['Baro_diff']=None # units Pa
launchesDF['6S_amp']=None # units mH20
launchesDF['6I_amp']=None # units mH20
launchesDF['6S_diff']=None #units mH20
launchesDF['6I_diff']=None #units mH20
launchesDF['6S_Pa']=None # units Pa
launchesDF['6I_Pa']=None # units Pa

for idx, eventrow in launchesDF.iterrows():
    # get start and end time. remember that spreadsheet is in UTC. well data are in local time but summer time is assumed throughout (UTC-4)
    launchtimeUTC = eventrow['datetime']

    stime = launchtimeUTC - pd.Timedelta(seconds=preseconds) - pd.Timedelta(hours=4)
    etime = launchtimeUTC + pd.Timedelta(seconds=eventseconds+postseconds) - pd.Timedelta(hours=4)
    welldayfile0 = None
    if stime.day != eventrow['datetime'].day:
        welldayfile0 = os.path.join(paths['welldata'],stime.strftime('%Y%m%d_final.pkl'))
        if not os.path.isfile(welldayfile0):
            welldayfile0 = None
    if welldayfile0:
        welldaydf0 = pd.read_pickle(welldayfile0)
    else:
        welldaydf0 = None
    welldayfile = os.path.join(paths['welldata'],launchtimeUTC.strftime('%Y%m%d_final.pkl'))
    if os.path.isfile(welldayfile):
        welldaydf = pd.read_pickle(welldayfile)
        if isinstance(welldaydf0, pd.DataFrame):
            welldaydf = pd.concat([welldaydf0, welldaydf])
        welldaydf = welldaydf.filter(regex='^(?!.*wldiff$)')
        print(f'Subsetting {os.path.basename(welldayfile)} from {stime} to {etime}')
        welldaydf['datetime'] = pd.to_datetime(welldaydf['datetime'])
        mask = (welldaydf['datetime']>=stime) & (welldaydf['datetime']<=etime)
        welldaydf = welldaydf.loc[mask]
        welldaydf = welldaydf.dropna(subset=['datetime'])

        if len(welldaydf)>0:
            # create event directory
            EVENTDIR = os.path.join(paths['EVENT_WAVEFORMS'], launchtimeUTC.isoformat())
            if not os.path.isdir(EVENTDIR):
                os.makedirs(EVENTDIR)

            channelsdf = pd.DataFrame(columns=['nslc', 'amplitude', 'DC_before', 'DC_after', 'DC_diff', 'units', 'corr_coef', 'shift'])

            # fix units
            for col in welldaydf.columns:
                if col in watercolumnsShallow + watercolumnsIntermediate:
                    welldaydf[col] = welldaydf[col] * 0.3048 # from ft_h20 to m_h20
                elif col in aircolumns:
                    welldaydf[col] = welldaydf[col] * 6894.76 # PSI to Pascals
            
            ax1 = welldaydf.plot(x='datetime', y=intersect(welldaydf, aircolumns), ylabel='Air Pressure (PSI)', title=f'Air Pressure for event at {launchtimeUTC}')
            plt.savefig(os.path.join(EVENTDIR, 'dataframe_aircolumns.png'))
            ax2 = welldaydf.plot(x='datetime', y=intersect(welldaydf, watercolumnsShallow), ylabel='NAVD88 (m)', title=f'Water Level in Shallow well for event at {launchtimeUTC}')
            plt.savefig(os.path.join(EVENTDIR, 'dataframe_watercolumns6S.png'))
            ax3 = welldaydf.plot(x='datetime', y=intersect(welldaydf, watercolumnsIntermediate), ylabel='NAVD88 (m)', title=f'Water Level in Intermediate well for event at {launchtimeUTC}')
            plt.savefig(os.path.join(EVENTDIR, 'dataframe_watercolumns6I.png'))
            
            st = dataframe2stream(welldaydf)
            best_trig = detectEvent(st)
            pad_secs = 10
            if len(best_trig)>0:    


                st3 = st.copy()
                st3.trim(starttime=best_trig['time']-pad_secs, endtime=best_trig['time']+pad_secs+best_trig['duration'])
                AP6S = st3.select(id='6S.Baro..HDD')
                bool_xcorr = False
                if len(AP6S)==1:
                    bool_xcorr = True
                    AP6S = AP6S.copy()[0].detrend()
                for channel in ['HDD', 'WLM']:
                    if channel=='HDD':
                        units = 'Pa'
                        titlestr = f'Air Pressure (Pa) for Event {launchtimeUTC}'
                    elif channel=='WLM':
                        units = 'm'
                        titlestr = f'Water Level (m) for Event {launchtimeUTC}'    
                    st4 = st3.select(channel=channel)

                    # time series plots on raw Stream
                    st4.plot(equal_scale=True, outfile=os.path.join(EVENTDIR, f'stream_{channel}.png'));


                    # we do the spectral processing on a detrended Stream
                    st5 = st4.copy()
                    st5.detrend('linear')
                    spobj = Spectrograms.icewebSpectrogram(stream=st5)
                    spobj = spobj.precompute()
                    spobj.plot(fmin=0.1, fmax=50.0, equal_scale=True, title=titlestr, outfile=os.path.join(EVENTDIR, f'sgram_{channel}.png'))
                    #st3.spectrogram()
                    spobj.compute_amplitude_spectrum(compute_bandwidth=True)
                    spobj.plot_amplitude_spectrum(normalize=True, title=titlestr)  
                    plt.savefig(os.path.join(EVENTDIR, f'spectrum_{channel}.png'))
                
                    for i,tr in enumerate(st5):
                        if bool_xcorr:
                            correlation = correlate(tr, AP6S, 100)
                            bestshift, bestvalue = xcorr_max(correlation, abs_max=True)
                        tr_before = st4[i].copy()
                        tr_before = tr_before.trim(starttime=best_trig['time']-pad_secs, endtime=best_trig['time']-pad_secs/2)
                        tr_after = st4[i].copy()
                        tr_after = tr_after.trim(starttime=best_trig['time']+best_trig['duration']+pad_secs/2, endtime=best_trig['time']+best_trig['duration']+pad_secs)
                        DC_before=np.median(tr_before.data)
                        DC_after=np.median(tr_after.data)
                        new_row = {'nslc':tr.id, 'amplitude':tr.max(), 'DC_before':DC_before, 'DC_after':DC_after, 'DC_diff':DC_after-DC_before, 'units':units}
                        #nErow['bw_min']=
                        #new_row['bw_max']=
                        new_row['corr_coef']=bestvalue
                        new_row['shift']=bestshift
                        channelsdf.loc[len(channelsdf)] = new_row
                display(channelsdf)
                ''' what statistics can we add to the dataframe for each event?
                * maximum absolute amplitude (from mean)
                * bandwidth?
                * detection on and off time
                * correlations between each trace - and a median correlation?
                * correlation best time lag between each trace 
                * DC level before and after
                * summary or event level (or per station) stats can go in the event catalog, more detailed per channel stats can go in the event
                '''

            if len(channelsdf)>0:
                # save channelsdf
                channelsdf.to_csv(os.path.join(EVENTDIR, 'channelsDF.csv'), index=False)

                # add content to launchesDF for this event
                airdf = channelsdf[channelsdf['nslc'].str.contains('.Baro.')]
                waterdf = channelsdf[channelsdf['nslc'].str.endswith('WLM')]
                water6S = waterdf[waterdf['nslc'].str.startswith('6S')]
                water6I = waterdf[waterdf['nslc'].str.startswith('6I')]
                launchesDF.at[idx, 'signalStrength'] = np.sum(best_trig['cft_peaks'])
                launchesDF.at[idx, 'Baro_amp'] = airdf['amplitude'].abs().mean(axis=0)
                launchesDF.at[idx,'Baro_diff'] = airdf['DC_diff'].mean(axis=0)
                launchesDF.at[idx, '6S_amp'] = water6S['amplitude'].abs().mean(axis=0)
                launchesDF.at[idx, '6I_amp'] = water6I['amplitude'].abs().mean(axis=0)
                launchesDF.at[idx, '6S_diff'] = water6S['DC_diff'].mean(axis=0)
                launchesDF.at[idx, '6I_diff'] = water6I['DC_diff'].mean(axis=0)
                

launchesDF['6S_Pa'] = launchesDF['6S_amp']*9806.65
launchesDF['6I_Pa'] = launchesDF['6I_amp']*9806.65
display(launchesDF)
launchesDF.to_csv(os.path.join(paths['EVENT_WAVEFORMS'], 'launchesDF.csv'))
launchesDF.plot(x='datetime',y=['Baro_amp', '6S_Pa', '6I_Pa'], style='o', ylabel='Pa', title='Comparing pressures in air, and in each well')
plt.savefig(os.path.join(EVENTDIR, 'catalog_pascals.png'))
display(launchesDF[['datetime', 'Baro_amp', '6S_Pa', '6I_Pa']])

Linux
{'HOME': '/home/thompsong', 'Developer': '/home/thompsong/Developer', 'repodir': '/home/thompsong/Developer/KSCRocketSeismoHydrology/Python/new_workflow', 'work': '/home/thompsong/work', 'local_outdir': '/home/thompsong/work/PROJECTS/KSC_EROSION', 'DROPBOX_TOP': '/home/thompsong/Dropbox', 'new_data': '/data/KSC/EROSION/fromdropboxinventory', 'DROPBOX_DATA_TOP': '/home/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3_Project_Documents/NASAprojects/201602_Rocket_Seismology/DATA/2022_DATA', 'dropbox_outdir': '/home/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3_Project_Documents/NASAprojects/201602_Rocket_Seismology/DATA/2022_DATA/new_workflow', 'WELLDATA_TOP': '/home/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3_Project_Documents/NASAprojects/201602_Rocket_Seismology/DATA/2022_DATA/WellData', 'TOB3_DIR': '/home/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3_Project_Documents/NASAprojects/201602_Rocket_Seismology/DATA/2022_DATA/WellData/Uploads', 'transducersCSVfile': '/home/thompsong/Developer/KSCRocke

,nslc,amplitude,DC_before,DC_after,DC_diff,units,corr_coef,shift
0,6S.12264.20.HDD,1452.903217,102180.848378,102176.674031,-4.174346,Pa,0.559397,1
1,6I.12264.29.HDD,1907.271597,102176.410401,102168.360176,-8.050225,Pa,0.350124,2
2,6S.Baro..HDD,1690.587631,102188.224021,102184.988946,-3.235075,Pa,0.999254,0
3,6I.Baro..HDD,1604.730716,102189.986216,102185.232234,-4.753982,Pa,0.785277,0
4,6S.12264.21.WLM,-0.058270,0.294703,0.295033,0.000330,m,-0.778895,0
5,6S.12264.19.WLM,-0.058282,0.292004,0.292334,0.000330,m,-0.779233,0
6,6I.21498.82.WLM,0.058084,0.274774,0.275104,0.000330,m,-0.766906,0
7,6I.21516.91.WLM,0.058003,0.274793,0.275124,0.000331,m,-0.771674,0
8,6S.12264.23.WLM,-0.058508,0.292120,0.292450,0.000330,m,-0.782950,0
9,6I.21516.92.WLM,-0.058242,0.272362,0.272693,0.000331,m,-0.777897,0


Subsetting 20220804_final.pkl from 2022-08-04 06:19:00 to 2022-08-04 06:49:00
4 Trace(s) in Stream:
6S.Baro..HDD    | 2022-08-04T06:19:00.000000Z - 2022-08-04T06:49:00.000000Z | 100.0 Hz, 180001 samples
6I.Baro..HDD    | 2022-08-04T06:19:00.000000Z - 2022-08-04T06:49:00.000000Z | 100.0 Hz, 180001 samples
6S.12264.23.WLM | 2022-08-04T06:19:00.000000Z - 2022-08-04T06:49:00.000000Z | 100.0 Hz, 180001 samples
6I.21516.92.WLM | 2022-08-04T06:19:00.000000Z - 2022-08-04T06:49:00.000000Z | 100.0 Hz, 180001 samples
{}
Subsetting 20220804_final.pkl from 2022-08-04 18:58:48 to 2022-08-04 19:28:48
4 Trace(s) in Stream:
6S.Baro..HDD    | 2022-08-04T18:58:48.000000Z - 2022-08-04T19:28:48.000000Z | 100.0 Hz, 180001 samples
6I.Baro..HDD    | 2022-08-04T18:58:48.000000Z - 2022-08-04T19:28:48.000000Z | 100.0 Hz, 180001 samples
6S.12264.23.WLM | 2022-08-04T18:58:48.000000Z - 2022-08-04T19:28:48.000000Z | 100.0 Hz, 180001 samples
6I.21516.92.WLM | 2022-08-04T18:58:48.000000Z - 2022-08-04T19:28:48.000000Z 

,nslc,amplitude,DC_before,DC_after,DC_diff,units,corr_coef,shift
0,6S.Baro..HDD,1428.592200,102163.316574,102160.272185,-3.044390,Pa,0.995455,0
1,6I.Baro..HDD,1530.804122,102128.489976,102123.637364,-4.852612,Pa,0.824857,0
2,6S.12264.23.WLM,0.058915,0.039259,0.039570,0.000311,m,-0.738029,0
3,6I.21516.92.WLM,0.058916,0.000701,0.001012,0.000311,m,-0.738028,0


Subsetting 20220819_final.pkl from 2022-08-19 15:11:20 to 2022-08-19 15:41:20


...

4 Trace(s) in Stream:
6S.Baro..HDD    | 2022-08-19T15:11:20.000000Z - 2022-08-19T15:41:20.000000Z | 100.0 Hz, 180001 samples
6I.Baro..HDD    | 2022-08-19T15:11:20.000000Z - 2022-08-19T15:41:20.000000Z | 100.0 Hz, 180001 samples
6S.12264.23.WLM | 2022-08-19T15:11:20.000000Z - 2022-08-19T15:41:20.000000Z | 100.0 Hz, 180001 samples
6I.21516.92.WLM | 2022-08-19T15:11:20.000000Z - 2022-08-19T15:41:20.000000Z | 100.0 Hz, 180001 samples
{}
Subsetting 20220828_final.pkl from 2022-08-27 23:31:20 to 2022-08-28 00:01:20
Subsetting 20220905_final.pkl from 2022-09-04 21:59:40 to 2022-09-04 22:29:40
10 Trace(s) in Stream:
6S.12264.21.WLM | 2022-09-04T21:59:40.000000Z - 2022-09-04T22:29:40.000000Z | 100.0 Hz, 180001 samples
6S.12264.19.WLM | 2022-09-04T21:59:40.000000Z - 2022-09-04T22:29:40.000000Z | 100.0 Hz, 180001 samples
6S.12264.20.HDD | 2022-09-04T21:59:40.000000Z - 2022-09-04T22:29:40.000000Z | 100.0 Hz, 180001 samples
6I.21498.82.WLM | 2022-09-04T21:59:40.000000Z - 2022-09-04T22:29:40.000000Z

,nslc,amplitude,DC_before,DC_after,DC_diff,units,corr_coef,shift
0,6S.12264.20.HDD,70.434159,101893.087745,101934.265932,41.178187,Pa,0.214736,-2
1,6I.12264.29.HDD,-194.325641,101898.727901,101897.862286,-0.865616,Pa,0.613170,-1
2,6S.Baro..HDD,564.454644,101886.573011,101882.719853,-3.853158,Pa,0.999543,0
3,6I.Baro..HDD,-799.700448,101885.678762,101882.706702,-2.972061,Pa,0.804710,0
4,6S.12264.21.WLM,-0.057614,0.403035,0.403428,0.000394,m,-0.998692,0
5,6S.12264.19.WLM,-0.057622,0.396906,0.397300,0.000394,m,-0.998957,0
6,6I.21498.82.WLM,-0.057613,0.394415,0.394809,0.000394,m,-0.998491,0
7,6I.21516.91.WLM,-0.057631,0.394980,0.395374,0.000394,m,-0.999182,0
8,6S.12264.23.WLM,-0.057641,0.400390,0.400784,0.000394,m,-0.999511,0
9,6I.21516.92.WLM,-0.057637,0.397651,0.398045,0.000394,m,-0.999383,0


Subsetting 20220919_final.pkl from 2022-09-18 20:08:40 to 2022-09-18 20:38:40
10 Trace(s) in Stream:
6S.12264.21.WLM | 2022-09-18T20:08:40.000000Z - 2022-09-18T20:38:40.000000Z | 100.0 Hz, 180001 samples
6S.12264.19.WLM | 2022-09-18T20:08:40.000000Z - 2022-09-18T20:38:40.000000Z | 100.0 Hz, 180001 samples
6S.12264.20.HDD | 2022-09-18T20:08:40.000000Z - 2022-09-18T20:38:40.000000Z | 100.0 Hz, 180001 samples
6I.21498.82.WLM | 2022-09-18T20:08:40.000000Z - 2022-09-18T20:38:40.000000Z | 100.0 Hz, 180001 samples
6I.21516.91.WLM | 2022-09-18T20:08:40.000000Z - 2022-09-18T20:38:40.000000Z | 100.0 Hz, 180001 samples
6I.12264.29.HDD | 2022-09-18T20:08:40.000000Z - 2022-09-18T20:38:40.000000Z | 100.0 Hz, 180001 samples
6S.Baro..HDD    | 2022-09-18T20:08:40.000000Z - 2022-09-18T20:38:40.000000Z | 100.0 Hz, 180001 samples
6I.Baro..HDD    | 2022-09-18T20:08:40.000000Z - 2022-09-18T20:38:40.000000Z | 100.0 Hz, 180001 samples
6S.12264.23.WLM | 2022-09-18T20:08:40.000000Z - 2022-09-18T20:38:40.000000Z

,nslc,amplitude,DC_before,DC_after,DC_diff,units,corr_coef,shift
0,6S.12264.20.HDD,54.295022,101659.796377,101655.655425,-4.140951,Pa,0.165401,3
1,6I.12264.29.HDD,-209.192432,101670.022809,101668.737543,-1.285266,Pa,0.623635,4
2,6S.Baro..HDD,1320.951490,101786.348387,101779.867312,-6.481074,Pa,0.999974,0
3,6I.Baro..HDD,703.742362,101787.865234,101781.246264,-6.618970,Pa,0.679795,0
4,6S.12264.21.WLM,0.058271,0.566852,0.566976,0.000124,m,-0.802067,0
5,6S.12264.19.WLM,0.057090,0.565666,0.565970,0.000304,m,-0.804645,0
6,6I.21498.82.WLM,-0.058856,0.553555,0.553604,0.000049,m,-0.787602,0
7,6I.21516.91.WLM,-0.058892,0.551298,0.551271,-0.000027,m,-0.793856,0
8,6S.12264.23.WLM,0.057715,0.565084,0.565327,0.000243,m,-0.816976,0
9,6I.21516.92.WLM,-0.059190,0.547501,0.547570,0.000069,m,-0.830561,0


Subsetting 20221008_final.pkl from 2022-10-08 18:55:00 to 2022-10-08 19:25:00
10 Trace(s) in Stream:
6S.12264.21.WLM | 2022-10-08T18:55:00.000000Z - 2022-10-08T19:25:00.000000Z | 100.0 Hz, 180001 samples
6S.12264.19.WLM | 2022-10-08T18:55:00.000000Z - 2022-10-08T19:25:00.000000Z | 100.0 Hz, 180001 samples
6S.12264.20.HDD | 2022-10-08T18:55:00.000000Z - 2022-10-08T19:25:00.000000Z | 100.0 Hz, 180001 samples
6I.21498.82.WLM | 2022-10-08T18:55:00.000000Z - 2022-10-08T19:25:00.000000Z | 100.0 Hz, 180001 samples
6I.21516.91.WLM | 2022-10-08T18:55:00.000000Z - 2022-10-08T19:25:00.000000Z | 100.0 Hz, 180001 samples
6I.12264.29.HDD | 2022-10-08T18:55:00.000000Z - 2022-10-08T19:25:00.000000Z | 100.0 Hz, 180001 samples
6S.Baro..HDD    | 2022-10-08T18:55:00.000000Z - 2022-10-08T19:25:00.000000Z | 100.0 Hz, 180001 samples
6I.Baro..HDD    | 2022-10-08T18:55:00.000000Z - 2022-10-08T19:25:00.000000Z | 100.0 Hz, 180001 samples
6S.12264.23.WLM | 2022-10-08T18:55:00.000000Z - 2022-10-08T19:25:00.000000Z